In [1]:
# Chapter16:Monte Carlo simulation
import random

In [2]:
# 16.1
# サイコロを投げた場合
def rollDie():
    return random.choice([1,2,3,4,5,6])

def checkPascal(numTrials):
    """numTrials: 試行回数 正の整数
       勝利する確率の評価値を表示する。"""
    numWins = 0
    for i in range(numTrials): # 試行回数分のループ
        for j in range(24): # 24回サイコロを振る
            d1 = rollDie() # 1つ目のサイコロを投げる
            d2 = rollDie() # 2つ目のサイコロを投げる
            if d1 == 6 and d2 == 6: # 両方で6が出た場合
                numWins += 1
                break # 24回投げる間にそろえば終了
    print("Probability of winning =", numWins/numTrials)

In [3]:
checkPascal(1000000)

Probability of winning = 0.491602


In [4]:
print('1-(35/36)^24 = ', 1-(35/36)**24)

1-(35/36)^24 =  0.4914038761309034


In [5]:
# 16.2 CrapsGameクラス
# class CrapsGame(object):
#     def __init__(self):
#         self.passWins, self.passLosses = 0,0 # 初期化
#         self.dpWins, self.dpLosses, self.dpPushes = 0,0,0 # 初期化
    
#     def playHand(self):
#         throw = rollDie() + rollDie() # 2つのサイコロの目を足す
#         if throw == 7 or throw == 11:
#             self.passWins += 1 # パスラインの場合勝利
#             self.dpLosses += 1 # どんとパスの場合負け
#         elif throw == 2 or throw == 3 or throw == 12:
#             self.passLosses += 1
#             if throw == 12:
#                 self.dpPushes += 1
#             else:
#                 self.dpWins += 1
#         else:
#             point = throw
#             while True:
#                 throw = rollDie() + rollDie()
#                 if throw == point:
#                     self.passWins += 1
#                     self.dpLosses += 1
#                     break
#                 elif throw == 7:
#                     self.passLosses += 1
#                     self.dpWins += 1
#                     break
#     def passResults(self):
#         return (self.passWins, self.passLosses)
    
#     def dpResults(self):
#         return (self.dpWins, self.dpLosses, self.dpPushes)

In [6]:
def variance(X):
    """Xを数のリストとする。
       Xの分散を返す"""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x-mean)**2
    return tot/len(X)

def stdDev(X):
    """を数のリストとする。
       Xの標準偏差を返す。"""
    return variance(X)**0.5

In [7]:
def crapsSim(handsPerGame, numGames):
    """handsPerGame: 1ゲーム当たりの手 1以上の正の整数
       handsPerGameの手からなるゲームをnumGames回プレイし、
       その結果を表示する。"""
    games = []

    # ゲームをnumGames回プレイする
    for t in range(numGames):
        c = CrapsGame()                 # インスタンス生成
        for i in range(handsPerGame):   # 1ゲーム当たり何回か手が行われる
            c.playHand()                # ゲーム結果を取得
        games.append(c)                 # ゲームのインスタンスをリストに格納

    # 各ゲームの統計値を求める
    pROIPerGame, dpROIPerGame = [],[] # p:パス, dp:ドントパス
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame)) # 投資収益率(Return On Investment)
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
    
    # 統計値の概要を求めて表示する
    meanROI = str(round((100*sum(pROIPerGame)/numGames),4)) + '%'
    sigma = str(round(100*stdDev(pROIPerGame),4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI,'Std Dev. =',sigma)
    meanROI = str(round((100*sum(dpROIPerGame)/numGames),4)) + '%'
    sigma = str(round(100*stdDev(dpROIPerGame),4)) + '%'
    print('Don\'t pass:', 'Mean ROI =', meanROI,'Std Dev. =',sigma)

In [8]:
crapsSim(20,10)

Pass: Mean ROI = 9.0% Std Dev. = 25.4755%
Don't pass: Mean ROI = -12.0% Std Dev. = 25.318%


In [9]:
# 標準偏差から95%信頼区間を求めてみる
print(5.0 - 2*20.6155, 5.0 + 2*20.6155)

-36.231 46.231


In [10]:
crapsSim(1000000,10)

Pass: Mean ROI = -1.3682% Std Dev. = 0.0704%
Don't pass: Mean ROI = -1.4087% Std Dev. = 0.0782%


In [11]:
crapsSim(20,100000)

Pass: Mean ROI = -1.4531% Std Dev. = 22.4071%
Don't pass: Mean ROI = -1.3214% Std Dev. = 22.0936%


### memo
- 標準偏差が大きい。20手のゲームを1回プレイした時の結果はとても不確実性が大きいことを示している。

In [12]:
# 16.3 性能を上げるために参照表を使う

In [14]:
# 16.4
class CrapsGame(object):
    def __init__(self):
        self.passWins, self.passLosses = 0,0 # 初期化
        self.dpWins, self.dpLosses, self.dpPushes = 0,0,0 # 初期化
    
    def passResults(self):
        return (self.passWins, self.passLosses)
    
    def dpResults(self):
        return (self.dpWins, self.dpLosses, self.dpPushes)

    def playHand(self):
        # playHandの、より高速な、もう1つの実装
        pointsDict = {4:1/3,5:2/5,6:5/11,8:5/11,9:2/5,10:1/3}
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
            self.passLosses += 1
            if throw == 12:
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            if random.random() <= pointsDict[throw]: # 7の前にポイントが出る
                self.passWins += 1
                self.dpLosses += 1
            else:
                self.passLosses += 1    # ポイントの前に7が出る
                self.dpWins += 1

In [16]:
crapsSim(1000,10)

Pass: Mean ROI = -1.24% Std Dev. = 2.1105%
Don't pass: Mean ROI = -1.27% Std Dev. = 2.149%


In [21]:
# 16.4 πを求める
def throwNeedles(numNeedles):
    inCircle = 0
    for Needles in range(1, numNeedles + 1):
        x = random.random() # 針を落とした時のx座標
        y = random.random() # 針を落とした時のy座標
        if (x*x + y*y)**0.5 <= 1:   # 原点からの距離が1以内かどうか
            inCircle += 1
    # 第1象限内の針のみを数えるため、4倍にする
    return 4*(inCircle/float(numNeedles))

def getEst(numNeedles,numTrials):
    estimates = []
    for t in range(numTrials):
        piGuess = throwNeedles(numNeedles)
        estimates.append(piGuess)
    sDev = stdDev(estimates)
    curEst = sum(estimates)/len(estimates)
    print('Est. =' + str(round(curEst,5)) + ',',\
            'Std. dev. =' + str(round(sDev,5)) + ','\
            'Needles =' + str(numNeedles))
    return(curEst,sDev)

def estPi(precision, numTrials):
    numNeedles = 1000
    sDev = precision
    while sDev >= precision / 1.96: # 95%信頼区間内に落ち着くまでループ
        curEst, sDev = getEst(numNeedles,numTrials)
        numNeedles *= 2 # 針の数を2倍に
    return curEst

In [22]:
estPi(0.01,100)

Est. =3.1428, Std. dev. =0.05786,Needles =1000
Est. =3.13908, Std. dev. =0.04274,Needles =2000
Est. =3.13897, Std. dev. =0.0262,Needles =4000
Est. =3.14223, Std. dev. =0.01917,Needles =8000
Est. =3.14109, Std. dev. =0.01278,Needles =16000
Est. =3.14188, Std. dev. =0.00816,Needles =32000
Est. =3.14204, Std. dev. =0.00639,Needles =64000
Est. =3.14123, Std. dev. =0.00446,Needles =128000


3.1412250000000013